In [15]:
import pandas as pd
import numpy as np
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package punkt to /Users/saahil/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/saahil/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
courses_df = pd.read_csv('/Users/saahil/Desktop/College/Sem 5/DBMS Lab/Project/coursera_courses.csv')

# job_df.head()

job_df = pd.read_csv('/Users/saahil/Desktop/College/Sem 5/DBMS Lab/Project/job_data_filtered.csv')
# courses_df.head()


In [17]:
job_df.head()

,job_title,category,company_name,city,state,country,job_description,job_type,is_remote
0,Full Stack Developer,Engineering,skilora,Remote,Karnataka,IN,We are currently looking for a Full Stack Deve...,Full-time,False
1,Windows Helpdesk Advisor,Computer,IBM,Bengaluru,Karnataka,IN,Introduction A Technical Support Representativ...,Full-time,False
2,CB-WLS- Credit Support Specialist Iv- Operatio...,Healthcare,"JPMorgan Chase Bank, N.A.",Bengaluru,Karnataka,IN,Job responsibilities: Review Loan documents an...,Full-time,False
3,Sales Executive-South Delhi,Part time,Rama Panels Pvt Ltd,Delhi,Delhi,IN,"Managing existing, Dealers & Sub dealers Devel...",Full-time|Fresher,False
4,Senior Software Engineer,IT,Larsen & Toubro Infotech Limited,Hinjewadi,Maharashtra,IN,"""Manual Tester Job Description Skills and Qual...",Full-time,False


In [18]:
courses_df.head()
courses_df['course_url'] = courses_df['course_url'].astype(str).str.slice(0,255)

In [19]:

# Initialize the stemmer
stemmer = PorterStemmer()

stop_words = set(stopwords.words('english'))
# Define a function to stem words in each course description
def stem_text(text):
    if isinstance(text, str):  # Check if the entry is a string
        words = nltk.word_tokenize(text)
        filtered_words = [word for word in words if word.lower() not in stop_words]
        return ' '.join(stemmer.stem(word) for word in filtered_words)
    else:
        return ""  # Return an empty string if it's not a valid string


# Apply the function to the course_description column
courses_df['course_description_stemmed'] = courses_df['course_description'].apply(stem_text)

courses_df.drop(columns = ['course_description'], inplace = True)

In [20]:
courses_df.head()

,course_title,course_organization,course_certificate_type,course_time,course_rating,course_reviews_num,course_difficulty,course_url,course_students_enrolled,course_skills,course_summary,course_description_stemmed
0,(ISC)² Systems Security Certified Practitioner...,ISC2,Specialization,3 - 6 Months,4.7,492,Beginner,https://www.coursera.org/specializations/sscp-...,"6,958","['Risk Management', 'Access Control', 'Asset',...",[],pursu better secur job opportun prove knowledg...
1,.NET FullStack Developer,Board Infinity,Specialization,1 - 3 Months,4.3,51,Intermediate,https://www.coursera.org/specializations/dot-n...,"2,531","['Web API', 'Web Development', 'Cascading Styl...",['Master .NET full stack web dev: from .NET co...,develop profici requir design develop comprehe...
2,21st Century Energy Transition: how do we make...,University of Alberta,Course,1 - 3 Months,4.8,62,Beginner,https://www.coursera.org/learn/21st-century-en...,"4,377",[],['Understand the complexity of systems supplyi...,"afford , abund reliabl energi fundament human ..."
3,A Crash Course in Causality: Inferring Causal...,University of Pennsylvania,Course,1 - 3 Months,4.7,517,Intermediate,https://www.coursera.org/learn/crash-course-in...,"39,004","['Instrumental Variable', 'Propensity Score Ma...",[],"heard phrase “ correl equal causation. ” , , e..."
4,A life with ADHD,University of Geneva,Course,1 - 3 Months,NaN,NaN,Beginner,https://www.coursera.org/learn/life-with-adhd,NaN,"['differential diagnosis and comorbidities', '...",[' Understand what ADHD is and the challenges ...,adhd challeng come ? whether person affect adh...


In [21]:
# job_df.head()
job_df['job_description_stemmed'] = job_df['job_description'].apply(stem_text)

In [22]:
job_df.head()

,job_title,category,company_name,city,state,country,job_description,job_type,is_remote,job_description_stemmed
0,Full Stack Developer,Engineering,skilora,Remote,Karnataka,IN,We are currently looking for a Full Stack Deve...,Full-time,False,current look full stack develop join team . sk...
1,Windows Helpdesk Advisor,Computer,IBM,Bengaluru,Karnataka,IN,Introduction A Technical Support Representativ...,Full-time,False,introduct technic support repres ibm front-lin...
2,CB-WLS- Credit Support Specialist Iv- Operatio...,Healthcare,"JPMorgan Chase Bank, N.A.",Bengaluru,Karnataka,IN,Job responsibilities: Review Loan documents an...,Full-time,False,job respons : review loan document process tra...
3,Sales Executive-South Delhi,Part time,Rama Panels Pvt Ltd,Delhi,Delhi,IN,"Managing existing, Dealers & Sub dealers Devel...",Full-time|Fresher,False,"manag exist , dealer & sub dealer develop new ..."
4,Senior Software Engineer,IT,Larsen & Toubro Infotech Limited,Hinjewadi,Maharashtra,IN,"""Manual Tester Job Description Skills and Qual...",Full-time,False,`` manual tester job descript skill qualif req...


In [23]:
def vectorise_text(df,col,lang):
  tfidf_vectorizer = TfidfVectorizer(stop_words=lang)
  tfidf_matrix = tfidf_vectorizer.fit_transform(df[col].fillna(''))
  cosine_sim_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)
  return cosine_sim_matrix

In [24]:
course_desc_vectorised = vectorise_text(courses_df,'course_description_stemmed','english')
job_desc_vectorised = vectorise_text(job_df,'job_description_stemmed','english')

: 

: 

In [ ]:
print(course_desc_vectorised)
print(job_desc_vectorised)

[[1.00000000e+00 3.33942017e-02 2.24338196e-02 ... 1.32452991e-03
  0.00000000e+00 0.00000000e+00]
 [3.33942017e-02 1.00000000e+00 4.91475347e-03 ... 5.60864879e-04
  0.00000000e+00 0.00000000e+00]
 [2.24338196e-02 4.91475347e-03 1.00000000e+00 ... 3.36843382e-04
  0.00000000e+00 0.00000000e+00]
 ...
 [1.32452991e-03 5.60864879e-04 3.36843382e-04 ... 1.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  1.00000000e+00 2.07607455e-02]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  2.07607455e-02 1.00000000e+00]]
[[1.         0.07300803 0.04910155 ... 0.0236295  0.034624   0.0855996 ]
 [0.07300803 1.         0.10492423 ... 0.02124363 0.14740748 0.0751814 ]
 [0.04910155 0.10492423 1.         ... 0.02340516 0.12971127 0.05902599]
 ...
 [0.0236295  0.02124363 0.02340516 ... 1.         0.0153556  0.02887163]
 [0.034624   0.14740748 0.12971127 ... 0.0153556  1.         0.04567877]
 [0.0855996  0.0751814  0.0590

In [ ]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(courses_df['course_description_stemmed'].fillna(''))


def recommend_courses(student_input, tfidf_matrix, courses_df):
    # Transform the student input into a TF-IDF vector using the same vectorizer
    student_input_vector = tfidf_vectorizer.transform([student_input])
    
    # Compute cosine similarity between the student input vector and all course descriptions
    similarity_scores = cosine_similarity(student_input_vector, tfidf_matrix).flatten()
    
    # Get indices of the top 5 highest similarity scores
    top_indices = similarity_scores.argsort()[-5:][::-1]
    
    print(top_indices)
    # Retrieve the recommended courses
    recommended_courses = courses_df.iloc[top_indices]
    
    return recommended_courses

In [ ]:
student_input = "I am interested in finance and marketing"

# Get the top 5 recommended courses
recommended_courses = recommend_courses(student_input, tfidf_matrix, courses_df)
print(recommended_courses[['course_title','course_url']])

[ 24 681 685 328 340]
                                  course_title  \
24           AWS Cloud Practitioner Essentials   
681                          Marketing Digital   
685        Marketing en redes sociales de Meta   
328      Finance for Non-Finance Professionals   
340  Financing and Investing in Infrastructure   

                                            course_url  
24   https://www.coursera.org/learn/aws-cloud-pract...  
681  https://www.coursera.org/learn/estrategia-mark...  
685  https://www.coursera.org/professional-certific...  
328  https://www.coursera.org/learn/finance-for-non...  
340  https://www.coursera.org/learn/infrastructure-...  


In [ ]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(courses_df['course_description_stemmed'].fillna(''))


def recommend_job(student_input, tfidf_matrix, job_df):
    # Transform the student input into a TF-IDF vector using the same vectorizer
    student_input_vector = tfidf_vectorizer.transform([student_input])
    
    # Compute cosine similarity between the student input vector and all course descriptions
    similarity_scores = cosine_similarity(student_input_vector, tfidf_matrix).flatten()
    
    # Get indices of the top 5 highest similarity scores
    top_indices = similarity_scores.argsort()[-5:][::-1]
    
    print(top_indices)
    # Retrieve the recommended courses
    recommended_jobs = job_df.iloc[top_indices]
    
    return recommended_jobs

student_input = "I am interested in finance and marketing"
recommended_jobs = recommend_job(student_input, tfidf_matrix, job_df)
recommended_jobs.drop(columns=['job_description_stemmed'],inplace=True)
print(f"{recommended_jobs}")



[ 24 681 685 328 340]
                                             job_title          category  \
24                          Talent Acquisition Partner        Facilities   
681           Regulatory Documentation Expert - Pharma  Customer Service   
685                        Assistant Manager (Service)        Government   
328                           Lobby Sales Officer CASA                PR   
340  Principal Engineer, Software Development Engin...       Engineering   

                company_name       city             state country  \
24                   skilora     Remote       Maharashtra      IN   
681                  corbion     Mumbai       Maharashtra      IN   
685  Matrix Weighing Systems     Rajkot           Gujarat      IN   
328                 Yes Bank      Solan  Himachal Pradesh      IN   
340          Western Digital  Bengaluru         Karnataka      IN   

                                       job_description   job_type  is_remote  
24   TELECOMMUTE LOCATION: 

/var/folders/w5/3b4wb27x71qcbmhl0jr0qzrc0000gn/T/ipykernel_96613/693508501.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommended_jobs.drop(columns=['job_description_stemmed'],inplace=True)
